In [18]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [19]:
df = pd.read_pickle('LSWMD.pkl')

In [20]:
df.head()

,waferMap,dieSize,lotName,waferIndex,trianTestLabel,failureType
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,1.0,[[Training]],[[none]]
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,2.0,[[Training]],[[none]]
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,3.0,[[Training]],[[none]]
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,4.0,[[Training]],[[none]]
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",1683.0,lot1,5.0,[[Training]],[[none]]


In [21]:
df = df.astype({"trianTestLabel": str, "failureType":str})

In [22]:
df["failureType"].unique()

array(["[['none']]", "[['Loc']]", '[]', "[['Edge-Loc']]", "[['Center']]",
       "[['Edge-Ring']]", "[['Scratch']]", "[['Random']]",
       "[['Near-full']]", "[['Donut']]"], dtype=object)

In [23]:
index_names = df[(df["failureType"] == "[['none']]") | (df["failureType"] == "[]")].index
df = df.drop(index_names)

In [24]:
df = df.reset_index(drop = True)

In [25]:
df["failureType"].unique()

array(["[['Loc']]", "[['Edge-Loc']]", "[['Center']]", "[['Edge-Ring']]",
       "[['Scratch']]", "[['Random']]", "[['Near-full']]", "[['Donut']]"],
      dtype=object)

In [26]:
wafer_map = list(df["waferMap"])

In [27]:
# # check shape distribute
# shapes = []
# for wafer in wafer_map:
#     shapes.append(wafer.shape[0])
#     shapes.append(wafer.shape[1])
# plt.hist(shapes, density=False, color = 'lightblue', cumulative = False, label = "shapes")

In [28]:
small_width = 25
small_height = 25

big_width = 50
big_height = 50

x_small = []
y_small = []

x_big = []
y_big = []
for i in range(len(wafer_map)):
    image = wafer_map[i]
    if(image.shape[0] >= big_height or image.shape[1] >= big_width):        
        resized = cv2.resize(wafer_map[i], (big_width, big_height), interpolation = cv2.INTER_AREA)
        resized = resized.reshape((resized.shape[0], resized.shape[1], 1))
        x_big.append(resized)
        y_big.append(df.loc[i, "failureType"])
    else:
        resized = cv2.resize(wafer_map[i], (small_width, small_height), interpolation = cv2.INTER_AREA)
        resized = resized.reshape((resized.shape[0], resized.shape[1], 1))
        x_small.append(resized)
        y_small.append(df.loc[i, "failureType"])
#     # if you want to use transfer learning
#     gray_three_channel = cv2.cvtColor(resized, cv2.COLOR_GRAY2BGR)
x_big = np.array(x_big)
x_small = np.array(x_small)
print(x_big.shape)
print(x_small.shape)

(7471, 50, 50, 1)
(18048, 25, 25, 1)


In [37]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn import preprocessing
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import layers, models
import seaborn
from sklearn.metrics import confusion_matrix

In [30]:
le_big = preprocessing.LabelEncoder()
y_big = le_big.fit_transform(y_big)
classes_num_big = len(le_big.classes_)
print(classes_num_big)

le_small = preprocessing.LabelEncoder()
y_small = le_small.fit_transform(y_small)
classes_num_small = len(le_small.classes_)
print(classes_num_small)

8
8


In [31]:
x_train_big, x_test_big, y_train_big, y_test_big = train_test_split(x_big, y_big, test_size=0.2, stratify = y_big, shuffle = True)
x_train_big, x_val_big, y_train_big, y_val_big = train_test_split(x_train_big, y_train_big, test_size=0.2, stratify = y_train_big, shuffle = True)
print(x_train_big.shape)
print(x_val_big.shape)
print(x_test_big.shape)

x_train_small, x_test_small, y_train_small, y_test_small = train_test_split(x_small, y_small, test_size=0.2, stratify = y_small, shuffle = True)
x_train_small, x_val_small, y_train_small, y_val_small = train_test_split(x_train_small, y_train_small, test_size=0.2, stratify = y_train_small, shuffle = True)
print(x_train_small.shape)
print(x_val_small.shape)
print(x_test_small.shape)

(4780, 50, 50, 1)
(1196, 50, 50, 1)
(1495, 50, 50, 1)
(11550, 25, 25, 1)
(2888, 25, 25, 1)
(3610, 25, 25, 1)


In [33]:
# # transfer learning
# base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape = (width, height, 3))
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# predictions = Dense(classes_num, activation='softmax')(x)

# model = Model(base_model.input, predictions)
# model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
# model.summary()
# hist = model.fit(x_train, y_train, batch_size = batch_size, epochs=epochs, validation_split = 0.2, shuffle = True)

In [38]:
epochs = 50
batch_size = 32
model_big = models.Sequential()
model_big.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(big_width, big_height, 1)))
model_big.add(MaxPooling2D((2, 2)))
model_big.add(BatchNormalization())
model_big.add(Dropout(0.2))
model_big.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_big.add(MaxPooling2D((2, 2)))
model_big.add(BatchNormalization())
model_big.add(Dropout(0.2))
model_big.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_big.add(MaxPooling2D((2, 2)))
model_big.add(BatchNormalization())
model_big.add(Dropout(0.2))
model_big.add(GlobalAveragePooling2D())
model_big.add(layers.Dense(classes_num_big, activation='softmax'))
model_big.summary()
model_big.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
hist = model_big.fit(x_train_big, y_train_big, batch_size = batch_size, epochs=epochs, validation_data = (x_val_big, y_val_big), shuffle = True)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 24, 24, 32)        128       
_________________________________________________________________
dropout_6 (Dropout)          (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 22, 22, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 64)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 11, 11, 64)       

Epoch 43/50
150/150 [==============================] - 1s 7ms/step - loss: 0.0389 - accuracy: 0.9902 - val_loss: 0.2082 - val_accuracy: 0.9373
Epoch 44/50
150/150 [==============================] - 1s 7ms/step - loss: 0.0511 - accuracy: 0.9824 - val_loss: 0.2118 - val_accuracy: 0.9406
Epoch 45/50
150/150 [==============================] - ETA: 0s - loss: 0.0397 - accuracy: 0.98 - 1s 7ms/step - loss: 0.0405 - accuracy: 0.9883 - val_loss: 0.4432 - val_accuracy: 0.9089
Epoch 46/50
150/150 [==============================] - 1s 7ms/step - loss: 0.0387 - accuracy: 0.9879 - val_loss: 0.3188 - val_accuracy: 0.9064
Epoch 47/50
150/150 [==============================] - 1s 7ms/step - loss: 0.0344 - accuracy: 0.9885 - val_loss: 0.2341 - val_accuracy: 0.9381
Epoch 48/50
150/150 [==============================] - 1s 7ms/step - loss: 0.0382 - accuracy: 0.9889 - val_loss: 0.2276 - val_accuracy: 0.9331
Epoch 49/50
150/150 [==============================] - 1s 7ms/step - loss: 0.0365 - accuracy: 0.9868

In [46]:
epochs = 50
batch_size = 32
model_small = models.Sequential()
model_small.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(small_width, small_height, 1)))
model_small.add(MaxPooling2D((2, 2)))
model_small.add(BatchNormalization())
model_small.add(Dropout(0.2))
model_small.add(layers.Conv2D(64, (3, 3), activation='relu'))
model_small.add(BatchNormalization())
model_small.add(Dropout(0.2))
model_small.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_small.add(BatchNormalization())
model_small.add(Dropout(0.2))
model_small.add(layers.Conv2D(256, (3, 3), activation='relu'))
model_small.add(BatchNormalization())
model_small.add(Dropout(0.2))
model_small.add(GlobalAveragePooling2D())
model_small.add(layers.Dense(classes_num_small, activation='softmax'))
model_small.summary()
model_small.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
hist = model_small.fit(x_train_small, y_train_small, batch_size = batch_size, epochs=epochs, validation_data = (x_val_small, y_val_small), shuffle = True)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_38 (Conv2D)           (None, 23, 23, 32)        320       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 11, 11, 32)        0         
_________________________________________________________________
dropout_37 (Dropout)         (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 9, 9, 64)          18496     
_________________________________________________________________
dropout_38 (Dropout)         (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 7, 7, 128)         73856     
_________________________________________________________________
dropout_39 (Dropout)         (None, 7, 7, 128)       

KeyboardInterrupt: 

In [ ]:
# loss, accuracy = model.evaluate(x_test, y_test)
# print(f"accuracy: {accuracy}")

In [ ]:
# array = confusion_matrix(y_test, model.predict_classes(x_test))
# df_cm = pd.DataFrame(array, index = [i for i in le.classes_], columns = [i for i in le.classes_])
# df_norm_col=(df-df.mean())/df.std()

# plt.figure(figsize = (10,7))
# seaborn.heatmap(df_cm, annot=True, fmt="d")